In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("theme_river_d3.csv");

In [18]:
df.head()

,Unnamed: 0,Position,Streams,Date,Region,Inv_Pos,Country
0,126486,113,35972,2017-09-22,FR,87,France
1,126767,194,23122,2017-09-23,FR,6,France
2,126956,183,19756,2017-09-24,FR,17,France
3,127147,174,21529,2017-09-25,FR,26,France
4,127327,154,23498,2017-09-26,FR,46,France


In [19]:
df['Date'] = pd.to_datetime(df['Date'])
r = pd.date_range(start=df.Date.min(), end=df.Date.max())
r

DatetimeIndex(['2017-09-20', '2017-09-21', '2017-09-22', '2017-09-23',
               '2017-09-24', '2017-09-25', '2017-09-26', '2017-09-27',
               '2017-09-28', '2017-09-29',
               ...
               '2018-05-25', '2018-05-26', '2018-05-27', '2018-05-28',
               '2018-05-29', '2018-05-30', '2018-05-31', '2018-06-01',
               '2018-06-02', '2018-06-03'],
              dtype='datetime64[ns]', length=257, freq='D')

In [20]:
countries = [x for _, x in df.groupby('Region')]
countries

[      Unnamed: 0  Position  Streams       Date Region  Inv_Pos  Country
 1599     2035615        36     7358 2017-09-22     AT      164  Austria
 1600     2035825        46     6315 2017-09-23     AT      154  Austria
 1601     2036024        45     5273 2017-09-24     AT      155  Austria
 1602     2036218        39     6045 2017-09-25     AT      161  Austria
 1603     2036423        44     5851 2017-09-26     AT      156  Austria
 1604     2036625        46     5877 2017-09-27     AT      154  Austria
 1605     2036826        47     5970 2017-09-28     AT      153  Austria
 1606     2037035        56     6192 2017-09-29     AT      144  Austria
 1607     2037234        55     5489 2017-09-30     AT      145  Austria
 1608     2037431        52     4550 2017-10-01     AT      148  Austria
 1609     2037629        50     5284 2017-10-02     AT      150  Austria
 1610     2037826        47     5770 2017-10-03     AT      153  Austria
 1611     2038020        41     6342 2017-10-04    

In [21]:
new_df = []
for country in countries:
    processed = country.set_index('Date').reindex(r, fill_value=country['Region'].values[0]).rename_axis('Date').reset_index()
    processed.loc[processed['Streams'] == processed['Region'], ['Streams']] = np.NaN
    processed['Streams'] = pd.to_numeric(processed['Streams'])
    processed.info()
    processed.index = pd.DatetimeIndex(processed.index)
    processed.interpolate(method='time', limit_direction='both' ,inplace=True)
    new_df.append(processed)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 7 columns):
Date          257 non-null datetime64[ns]
Unnamed: 0    257 non-null object
Position      257 non-null object
Streams       74 non-null float64
Region        257 non-null object
Inv_Pos       257 non-null object
Country       257 non-null object
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 14.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 7 columns):
Date          257 non-null datetime64[ns]
Unnamed: 0    257 non-null object
Position      257 non-null object
Streams       233 non-null float64
Region        257 non-null object
Inv_Pos       257 non-null object
Country       257 non-null object
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 14.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 7 columns):
Date          257 non-null datetime64[ns]
Unnamed: 

In [22]:
new_df = pd.concat(new_df, axis=0)


In [23]:
new_df.to_csv("interpolated_rank.csv")

In [24]:
new_df.reset_index()
final_df = new_df.drop(['Unnamed: 0', 'Position', "Inv_Pos"], axis=1)
final_df.reset_index().to_csv("interpolated_rank_stripped.csv")

In [25]:
from sklearn import preprocessing

x = final_df.Streams.values #returns a numpy array
x = x.reshape(-1,1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
final_df['Streams_norm'] = x_scaled

In [27]:
final_df.reset_index().drop(['index', 'Streams', 'Country'],axis=1).to_csv("interpolated_rank_stripped.csv", index=False)